In [69]:
type direction = Left | Right
type state = string

type direction = Left | Right


type state = string


In [70]:
module type TAPE = sig
  type t

  val make : string -> t
  val move : direction -> t -> t
  val read : t -> char
  val write : char -> t -> t
  val print : t -> unit
end

module type TAPE =
  sig
    type t
    val make : string -> t
    val move : direction -> t -> t
    val read : t -> char
    val write : char -> t -> t
    val print : t -> unit
  end


In [71]:
module Tape : TAPE = struct
  type t = {left : char list ; right : char list}
  let string_to_char_list s =
    List.of_seq (String.to_seq s)

  let make s =
    {left  = []; right = string_to_char_list s}

  let move (d : direction) (tape : t) =
    match d with
    | Left -> (match tape.left with
      | [] -> {left = []; right = ' ' :: tape.right}
      | x :: xs -> {left = xs; right = x :: tape.right})
    | Right -> (match tape.right with
      | [] -> {left = ' ' :: tape.left ; right = []}
      | x :: xs -> {left = x :: tape.left ; right = xs})

  let read (tape : t) =
    match tape.right with
    | [] -> ' '
    | x :: xs -> x

  let write (x : char) (tape : t) =
    match tape.right with
    | [] -> {left = tape.left; right = [x]}
    | y :: ys -> {left = tape.left; right = x :: ys }
  
  let print (tape : t) =
    let left_str = String.concat "" (List.rev_map (String.make 1) tape.left) in
    let right_str = String.concat "" (List.map (String.make 1) tape.right) in
    let full_tape = left_str ^ right_str in
    let head_position = String.length left_str in
    let marker = String.make head_position ' ' ^ "^" in
    print_endline (full_tape ^ "\n" ^ marker)

end

module Tape : TAPE


In [72]:
Tape.(
  make "ABCDE"
  |> move Right
  |> move Right
  |> write '!'
  |> move Left
  |> print
)

AB!DE
 ^


- : unit = ()


In [73]:
module type MACHINE = sig
  type t
  val make : state -> state list -> t
  val initial : t -> state
  val add_transition : state -> char -> state -> char -> direction -> t -> t
  val step : t -> state -> Tape.t -> (state * Tape.t) option
end

module type MACHINE =
  sig
    type t
    val make : state -> state list -> t
    val initial : t -> state
    val add_transition :
      state -> char -> state -> char -> direction -> t -> t
    val step : t -> state -> Tape.t -> (state * Tape.t) option
  end


In [1]:
module Machine : MACHINE = struct
  type t = {state_list : state list ; initial : state ; delta : (state * char * direction) option array array; state_index : state -> int }
  module StateMap = Map.Make(String)

  let create_hash_table strings =
    let table = Hashtbl.create (List.length strings) in
    List.iteri (fun i s -> Hashtbl.add table s i) strings;
    table
  (* Opis delovanja stroja: za prehodno funkcijo uporabmo matriko, ki indeksu stanja in characterja priredi ustrezno vrednost
  Za hitrejšo učinkovitost stanja shranjujemo kot integerje, saj lahko tako namesto uporabe hash tabele, da pridobimo indeks nekega stanja,
  uporabimo direknto funkcijo int_of_string, ki je hitrejša. To dosežemo tako, da vedno ko pri funkciji make oziroma add_transition dodamo v stroj neka stanja,
  le-teh ne dodamo kot dejansko ime stanja, ampak jih najprej s hash tabelo pretvorimo v njihov int in nato ta dodamo string_of_int za ta indeks.*)

  let make initial_state states =
    let all_states = initial_state :: states in
    let num_symbols = 256 in
    let num_states = List.length all_states in
    let state_table = create_hash_table all_states in
    let state_to_int str =
      Hashtbl.find state_table str in
    let transitions = Array.make_matrix num_states num_symbols None in 
    {state_list = all_states; initial = string_of_int (state_to_int initial_state); delta = transitions; state_index = state_to_int}

  let initial t =
    t.initial
  
  let add_transition input_state input_char output_state output_char output_direction tm =
    let s_index = tm.state_index input_state in
    let char_index = int_of_char input_char in
    let outs_index_str = string_of_int (tm.state_index output_state) in
    let kopija = Array.copy tm.delta in
    kopija.(s_index).(char_index) <- Some (outs_index_str, output_char, output_direction);
    {state_list = tm.state_list ; initial = tm.initial; delta = kopija ; state_index = tm.state_index} 
  
  let step tm state tape =
    let s_index = int_of_string state in
    let head = Tape.read tape in
    let char_index = int_of_char head in
    match tm.delta.(s_index).(char_index) with
    | None -> None
    | Some (output_state, output_char, output_direction) -> Some (output_state, Tape.move output_direction (Tape.write output_char tape))

end

error: compile_error

In [75]:
let binary_increment =
  Machine.(
    make "right" [ "carry"; "done" ]
    |> add_transition "right" '1' "right" '1' Right
    |> add_transition "right" '0' "right" '0' Right
    |> add_transition "right" ' ' "carry" ' ' Left
    |> add_transition "carry" '1' "carry" '0' Left
    |> add_transition "carry" '0' "done" '1' Left
    |> add_transition "carry" ' ' "done" '1' Left
  )

val binary_increment : Machine.t = <abstr>


In [76]:
let slow_run (tm : Machine.t) str = 
  let initial_tape = Tape.make str in
  Tape.print initial_tape ;
  print_endline (Machine.initial tm);
  let rec aux t s tape =
    match Machine.step t s tape with 
    | None -> print_endline("Machine halted") ;
    | Some (s1, tape1) -> 
      Tape.print tape1 ;
      print_endline(s1) ;
      aux t s1 tape1
    in
  aux tm (Machine.initial tm) initial_tape


val slow_run : Machine.t -> string -> unit = <fun>


In [77]:
let speed_run (tm : Machine.t) str = 
  let initial_tape = Tape.make str in
  let rec aux t s tape n =
    match Machine.step t s tape with 
    | None -> Tape.print tape;
              print_endline(string_of_int n)
    | Some (s1, t1) ->
      aux t s1 t1 (n + 1)
    in
  aux tm (Machine.initial tm) initial_tape 1

val speed_run : Machine.t -> string -> unit = <fun>


In [78]:
let test_run (tm : Machine.t) str n= 
  let initial_tape = Tape.make str in
  Tape.print initial_tape ;
  print_endline (Machine.initial tm);
  let rec aux turing state tape m =
    match m with 
    | 0 -> print_endline "X"
    | m ->  match Machine.step turing state tape with 
    | None -> print_endline ("HALTS") ;
    | Some (s1, t1) -> 
      Tape.print t1 ;
      print_endline (s1);
      aux turing s1 t1 (m - 1)
    in 
    aux tm (Machine.initial tm) initial_tape n

val test_run : Machine.t -> string -> int -> unit = <fun>


In [79]:
let primer_slow_run =
  slow_run binary_increment "1011" 

1011
^
0
1011
 ^
0
1011
  ^
0
1011
   ^
0
1011
    ^
0
1011 
   ^
1
1010 
  ^
1
1000 
 ^
1
1100 
^
2
Machine halted


val primer_slow_run : unit = ()


In [80]:
let primer_speed_run =
  speed_run binary_increment "1011"

1100 
^
9


val primer_speed_run : unit = ()


In [81]:
let for_state (s : state) (instructions : (state -> Machine.t -> Machine.t) list list) (tm : Machine.t) =
  let all_functions = List.concat instructions in 
  List.fold_left (fun acc f -> f s acc) tm all_functions

let for_character (c : char) (f : char -> state -> Machine.t -> Machine.t) =
  [f c]
  
let for_characters (chars : string) (f : char -> state -> Machine.t -> Machine.t) =
  List.init (String.length chars) (fun i -> f (String.get chars i))
  
let write_and_move (c : char) (d : direction) =
  fun in_char state -> Machine.add_transition state in_char state c d
  
let move (d : direction) =
  fun c state ->
    Machine.add_transition state c state c d
      
let switch_and_move (s : state) (d : direction) =
  fun c in_state -> 
    Machine.add_transition in_state c s c d
  
let write_switch_and_move (c : char) (s : state) (d : direction) =
  fun in_char in_state ->
    Machine.add_transition in_state in_char s c d



val for_state :
  state ->
  (state -> Machine.t -> Machine.t) list list -> Machine.t -> Machine.t =
  <fun>


val for_character :
  char ->
  (char -> state -> Machine.t -> Machine.t) ->
  (state -> Machine.t -> Machine.t) list = <fun>


val for_characters :
  string ->
  (char -> state -> Machine.t -> Machine.t) ->
  (state -> Machine.t -> Machine.t) list = <fun>


val write_and_move :
  char -> direction -> char -> state -> Machine.t -> Machine.t = <fun>


val move : direction -> char -> state -> Machine.t -> Machine.t = <fun>


val switch_and_move :
  state -> direction -> char -> state -> Machine.t -> Machine.t = <fun>


val write_switch_and_move :
  char -> state -> direction -> char -> state -> Machine.t -> Machine.t =
  <fun>


In [82]:
let binary_increment' =
  Machine.make "right" ["carry"; "done"]
  |> for_state "right" [
    for_characters "01" @@ move Right;
    for_character ' ' @@ switch_and_move "carry" Left
  ]
  |> for_state "carry" [
    for_character '1' @@ write_and_move '0' Left;
    for_characters "0 " @@ write_switch_and_move '1' "done" Left
  ] 

val binary_increment' : Machine.t = <abstr>


In [83]:
let primer_speed_run2 =
  speed_run binary_increment' "101011"

101100 
  ^
11


val primer_speed_run2 : unit = ()


In [84]:
let busy_beaver4 = 
  Machine.(make "A" ["B"; "C"; "D"]
  |> add_transition "A" ' ' "B" '1' Right
  |> add_transition "A" '1' "B" '1' Left
  |> add_transition "B" ' ' "A" '1' Left
  |> add_transition "B" '1' "C" ' ' Left
  |> add_transition "C" '1' "D" '1' Left
  |> add_transition "D" ' ' "D" '1' Right
  |> add_transition "D" '1' "A" ' ' Right
  )

val busy_beaver4 : Machine.t = <abstr>


In [85]:
let primer_bb4 = 
  speed_run busy_beaver4 ""

  111111111111
^
107


val primer_bb4 : unit = ()


In [86]:
let busy_beaver5 =
  Machine.(make "A" ["B"; "C"; "D"; "E"]
  |> add_transition "A" ' ' "B" '1' Right
  |> add_transition "A" '1' "C" '1' Left
  |> add_transition "B" ' ' "C" '1' Right
  |> add_transition "B" '1' "B" '1' Right
  |> add_transition "C" ' ' "D" '1' Right
  |> add_transition "C" '1' "E" ' ' Left
  |> add_transition "D" ' ' "A" '1' Left
  |> add_transition "D" '1' "D" '1' Left
  |> add_transition "E" '1' "A" ' ' Left
)


val busy_beaver5 : Machine.t = <abstr>


In [87]:
let primer_bb5 = 
  speed_run busy_beaver5 ""

  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 

val primer_bb5 : unit = ()


In [88]:
let kr_neki n =
  let create_hash_table strings =
    let table = Hashtbl.create (List.length strings) in
    List.iteri (fun i s -> Hashtbl.add table s i) strings;
    table
  in
  let index tbl str =
    Hashtbl.find tbl str
  in
  let all_states = ["A"; "B"; "C"; "D"; "E"] in
  let state_table = create_hash_table all_states in
  let state_to_int str =
    index state_table str in
  let rec aux m acc =
    match m with 
    | 0 -> 0
    | x -> aux (x - 1) (state_to_int "C")
  in 
  aux n 0

val kr_neki : int -> int = <fun>


In [89]:
kr_neki 50000000

- : int = 0


In [90]:
(* let make_state_to_int_func (states : string list) : (string -> int) =
  let state_map = 
    List.fold_left (fun map (i, state) -> StateMap.add state i map) StateMap.empty (List.mapi (fun i state -> (i, state)) states)
  in
  fun state -> StateMap.find state state_map *)

In [91]:
module StringMap = Map.Make(String)

let create_string_to_int_map strings =
  List.fold_left (fun acc (i, str) -> StringMap.add str i acc)
    StringMap.empty
    (List.mapi (fun i str -> (i, str)) strings)



module StringMap :
  sig
    type key = String.t
    type 'a t = 'a Map.Make(String).t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val filter_map : (key -> 'a -> 'b option) -> 'a t -> 'b t
    val partition : (key -> 'a -> 

val create_string_to_int_map : StringMap.key list -> int StringMap.t = <fun>


In [92]:
let string_to_int_func strings =
  let string_to_int_map = create_string_to_int_map strings in
  fun str ->
    try StringMap.find str string_to_int_map
    with Not_found -> -1 (* Default value for unmapped strings *)

let aa str = 
    string_to_int_func ["A"] str

val string_to_int_func : StringMap.key list -> StringMap.key -> int = <fun>


val aa : StringMap.key -> int = <fun>


In [93]:
let kr_neki2 n =
  let all_states = ["A"; "B"; "C"; "D"; "E"] in
  let aa str = 
    string_to_int_func all_states str in
  let rec aux m acc =
    match m with 
    | 0 -> 0
    | x -> aux (x - 1) (int_of_string "11")
  in 
  aux n 0

File "[93]", line 3, characters 6-8:
3 |   let aa str = 
          ^^
Warning 26 [unused-var]: unused variable aa.


val kr_neki2 : int -> int = <fun>


In [94]:
kr_neki2 50000000

- : int = 0
